# ABBA cell count analysis

This notebook is the last step in the ABBA whole-brain cell counting analysis.  
It assumes you have done the following steps:
- Alignment of brain slices in ABBA, exported to a QuPath project.
- Detected cells of interest in QuPath. The detections should be exported to ```.csv``` files (one per slice) in a folder called ```results```. 
- If there are regions to exclude, you should have drawn them and exported to ```.txt``` files (one per slice) in a folder called ```regions_to_exclude```.

Run this notebook to load the cell counts and do analysis on them. 

## Before we start ...
### Set parameters

In [ ]:
import os

CONFIG_FILE_NAME = "braian_config.toml"                     # assumes the file is in DATA_ROOT directory
# EXPERIMENT_DIRECTORY = "p6"
# EXPERIMENT_DIRECTORY = "experiment"
# EXPERIMENT_DIRECTORY = "proof"
# EXPERIMENT_DIRECTORY = "rebased_on_mjd"
# EXPERIMENT_DIRECTORY = "ieg"
EXPERIMENT_DIRECTORY = "cohort4iba1"
# EXPERIMENT_DIRECTORY = "p17iba1"
# EXPERIMENT_DIRECTORY = "cohort4gaba"
# EXPERIMENT_DIRECTORY = "p17gaba"

USE_REMOTE_DATA = False                                     # if True, it tries to read the data on the laboratory's server
# ###################################### REMOTE DIRECTORIES #####################################
IS_COLLABORATION_PROJ = False
COLLABORATION_DIRECTORY = os.path.join("Mathias Schmidt", "soumnya")

# ###################################### LOCAL DIRECTORIES ######################################
# DATA_ROOT  = f"../data/experiments/sowmya/{EXPERIMENT_DIRECTORY}"
DATA_ROOT  = f"../data/experiments/{EXPERIMENT_DIRECTORY}"
PLOTS_ROOT = f"../plots/{EXPERIMENT_DIRECTORY}"

In [ ]:
REMOVE_SMALL_REGIONS_FROM_SLICES = False
REMOVE_HIGH_CV_REGIONS = False
CVAR_THRESHOLD = 1

# ###################################### PLOT OPTIONS ######################################
PLOT_ALLENBRAIN_HIERARCHY = False
PLOT_ANIMALS_ROOTS = True
PLOT_COEFFICIENT_OF_VARIATION = True
PLOT_COEFFICIENT_OF_VARIATION_THRESHOLD = 1

SAVE_ANIMALS = True
SAVE_GROUPS = True

# Script's code

In [ ]:
import os
import sys
from typing import List

project_path = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(project_path)
import BraiAn

In [ ]:
if USE_REMOTE_DATA:
    DATA_ROOT, PLOTS_ROOT = BraiAn.remote_dirs(EXPERIMENT_DIRECTORY, IS_COLLABORATION_PROJ, COLLABORATION_DIRECTORY)

data_input_path = os.path.join(DATA_ROOT, "QuPath_output")
data_output_path= os.path.join(DATA_ROOT, "BraiAn_output")
config_file = os.path.join(DATA_ROOT, CONFIG_FILE_NAME)

if not(os.path.exists(data_output_path)):
    os.makedirs(data_output_path, exist_ok=True)
os.makedirs(PLOTS_ROOT, exist_ok=True)

In [ ]:
import tomllib

with open(config_file, "rb") as f:
    config = tomllib.load(f)
config
# ######################################### LOAD CONFIG #########################################
EXPERIMENT_NAME = config["experiment"]["name"]

ATLAS_VERSION = config["atlas"]["version"]
BRANCHES_TO_EXCLUDE = config["atlas"]["excluded-branches"]
USE_LITERATURE_REUNIENS =  config["atlas"]["use-literature-reuniens"]   # add a 'REtot' region, merging the following regions: 'RE', 'Xi', 'RH'

BRAINS_AREA_KEY = config["brains"]["area-column"]
BRAINS_TRACER_KEYS = config["brains"]["tracer-columns"]
BRAINS_OVERLAPPING_TRACERS = [[i_marker-1 for i_marker in comp["markers"]]
                              for comp in config["comparison"].values()
                              if isinstance(comp, dict) and comp["metric"] == "Overlapping" and "markers" in comp]
BRAINS_MARKERS = config["brains"]["markers"]
BRAINS_AGGREGATION_MODE = config["brains"]["slices-aggregation-mode"]   # available options are: 'sum', 'mean'/'avg', 'std', 'variation'/'cvar'

REGIONS_TO_PLOT_SELECTION_METHOD = config["comparison"]["regions-to-plot"]  # Available options are:
                                                                            #   - "summary structures"
                                                                            #   - "major divisions"
                                                                            #   - "depth <n>" where <n> is an integer of the depth desired
                                                                            #   - "structural level <n>" where <n> is an integer of the level desired
                                                                            #   - "smallest"
                                                                            # where <n> is an integer of the depth/level desired

from collections import namedtuple
GroupDirectory = namedtuple("GroupDirectory", "id name dirs")
groups = [
    GroupDirectory(
        id=int(group[len("group"):])-1,
        name=config["experiment"][group]["name"],
        dirs=config["experiment"][group]["dirs"]
    ) for group in config["experiment"] if group.startswith("group") and group[len("group"):].isdigit()
]

## The Allen Brain Atlas

We start by importing the mouse Allen Brain Atlas, in which we find information about all brain regions (their parent region and children regions in the brain hierarchy, for example).

In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = os.path.join(project_path, "data", "AllenMouseBrainOntology.json")
AllenBrain = BraiAn.AllenBrainHierarchy(path_to_allen_json, BRANCHES_TO_EXCLUDE, use_literature_reuniens=USE_LITERATURE_REUNIENS, version=ATLAS_VERSION)

#parent_region = AllenBrain.parent_region
#direct_subregions = AllenBrain.direct_subregions
#full_name = AllenBrain.full_name
#regions = AllenBrain.list_all_subregions("root", mode="depth")

In [ ]:
match REGIONS_TO_PLOT_SELECTION_METHOD:
    case "summary structures":
        # selects the Summary Strucutures
        path_to_summary_structures = os.path.join(project_path, "data", "AllenSummaryStructures.csv")
        AllenBrain.select_from_csv(path_to_summary_structures, include_nre_tot=USE_LITERATURE_REUNIENS)
    case "major divisions":
        AllenBrain.select_regions(BraiAn.MAJOR_DIVISIONS)
    case s if s.startswith("smallest"):
        AllenBrain.select_leaves()
    case s if s.startswith("depth"):
        n = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")[-1]
        try:
            depth = int(n)
        except Exception:
            raise Exception("Could not retrieve the <n> parameter of the 'depth' method for 'REGIONS_TO_PLOT_SELECTION_METHOD'")
        AllenBrain.select_at_depth(depth)
    case s if s.startswith("structural level"):
        n = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")[-1]
        try:
            level = int(n)
        except Exception:
            raise Exception("Could not retrieve the <n> parameter of the 'structural level' method for 'REGIONS_TO_PLOT_SELECTION_METHOD'")
        AllenBrain.select_at_structural_level(level)
    case _:
        raise Exception(f"Invalid value '{REGIONS_TO_PLOT_SELECTION_METHOD}' for REGIONS_TO_PLOT_SELECTION_METHOD")
selected_regions = AllenBrain.get_selected_regions()
print(f"You selected {len(selected_regions)} regions to plot.")

We can also visualize the hierarchy of brain regions as a network (a tree). **Note that running the above cell may take a few minutes**.

In [ ]:
## Plot brain region hierarchy
## If you want to plot it, install PyDot (pydot)
if PLOT_ALLENBRAIN_HIERARCHY:
    fig = AllenBrain.plot_plotly_graph()
    fig.show()

Based on the graph above, you might want to specify the regions on which you want to do further PLS analysis:  
*Note: to see more information about the regions, hover over them with your mouse.*

- Specify a level. Analysis can only be done on one level (slice) in the brain region.

- To exclude brain regions that belong to a certain branch, add the *abbreviated* nodes at the beginning of the branches to the list above.  
Example:  
```branches_to_exclude = ["retina", "VS"]```  
means that **all the subregions that belong to the retina and the ventricular systems** are excluded from the PLS analysis.

## Load data

Now, we're ready to read the ```.csv``` files with the cell counts, and also the exclusion files (if there were regions to exclude).  
Below, you have to specify:
- ```animals_root```: Absolute path to the folder that contains the animal folders.
- ```group_1_dirs```: A list of names of the folders corresponding to animals in **Group 1** (e.g., Control group). Indeed, it is necessary to store the results in individual folders for each animal.
- ```group_2_dirs```: A list of names of the folders corresponding to animals in **Group 2** (e.g., Stress group).
- ```group_1_name```: A meaningful string for Group 1.
- ```group_2_name```: A meaningful string for Group 2.
- ```area_key```: A string of the column in the ```.csv``` files that refers to the size of a brain areatra
- ```tracer_key```: A string of the column in the ```.csv``` files that refers to the tracer number used to highlight the marker
- ```marker```: A string of the marker we would like to highlight (e.g. CFos)

Provare a modificar per ottenere densita in mm^2 (da micron)

Now, we load the Control and Stress results seperately in two pandas dataframes, and save the results.

**Note**: regions to exclude are automatically excluded.

In [ ]:
import re

def fix_overlap_detection_if_old_qpscript(sliced_brain: BraiAn.SlicedBrain):
    # if sliced_brain has was computed on data collected from an old QuPath script,
    # then the number of detection of the first marker is ~wrong. It must be summed to the overlaps between marker1 and marker2
    for i in range(len(sliced_brain.markers)): # e.g. header="GABA-(cFos+GABA)"
        marker1_diff = sliced_brain.markers[i]
        # see https://regex101.com/r/LLwGIl/1
        markers = re.compile("(?P<m1>\w+)-\((?:(\w+)\+(?P=m1)|(?P=m1)\+(\w+))\)").findall(marker1_diff)
        # e.g markers=[('GABA', 'cFos', '')]
        if len(markers) == 0:
            continue
        markers = [m for m in markers[0] if len(m) != 0]
        marker1, marker2 = markers  # e.g. marker1="GABA" and marker2="cFos"
        for brain_slice in sliced_brain.slices:
            brain_slice.data[marker1_diff] += brain_slice.data[f"{marker1_diff}+{marker2}"]
            brain_slice.data.rename(columns={marker1_diff: marker1, f"{marker1_diff}+{marker2}": f"{marker1}+{marker2}"}, inplace=True)
        sliced_brain.markers[i] = marker1
        overlap_i = next(i for i in range(len(sliced_brain.markers)) if sliced_brain.markers[i] == f"{marker1_diff}+{marker2}")
        sliced_brain.markers[overlap_i] = f"{marker1}+{marker2}"
        

In [ ]:
groups_slices: List[List[BraiAn.SlicedBrain]] = []

for i in range(len(groups)):
    group_slices = []
    for animal_dir in groups[i].dirs:
        if not os.path.isdir(os.path.join(data_input_path, animal_dir)):
            print(f"WARNING: could not find the directory '{animal_dir}' in '{EXPERIMENT_DIRECTORY}'. Skipping this animal.")
            continue
        sliced_brain = BraiAn.SlicedBrain(animal_dir,
                                            os.path.join(data_input_path, animal_dir),
                                            AllenBrain,
                                            BRAINS_AREA_KEY,
                                            BRAINS_TRACER_KEYS,
                                            BRAINS_MARKERS,
                                            *BRAINS_OVERLAPPING_TRACERS,
                                            area_units="µm2")
        fix_overlap_detection_if_old_qpscript(sliced_brain)
        group_slices.append(sliced_brain)
    groups_slices.append(group_slices)
    print(f"Imported all brain slices from {str(len(groups[i].dirs))} animals of {groups[i].name} group.")

In [ ]:
if REMOVE_SMALL_REGIONS_FROM_SLICES:
    for group_ in groups_slices:
        for animal_ in group_:
            for s in animal_.slices:
                s._data = s.data
                # TODO: currently there is no differentiation between real markers and overlapping markers.
                # This bad workaround excludes all those markers having a '+' in the name.
                real_markers = [m for m in animal_.markers if "+" not in m]
                s.data = s.data[(s.data[real_markers] != 1).any(axis=1) & (s.data.area > 0.001)].copy(deep=True)

In [ ]:
if PLOT_ANIMALS_ROOTS:
    region_name = "root"
    root_plot = BraiAn.plot_region_density(region_name, *groups_slices, width=1000, height=500)
    root_plot.show()

In [ ]:
# print("N regions above threshold:", sum([(brain.data > cv_threshold).sum() for brain in cvar_brains]))
# print("N regions below threshold:", sum([(brain.data <= cv_threshold).sum() for brain in cvar_brains]))
if PLOT_COEFFICIENT_OF_VARIATION:
    cvar_plot = BraiAn.plot_cv_above_threshold(AllenBrain, *groups_slices, cv_threshold=PLOT_COEFFICIENT_OF_VARIATION_THRESHOLD, width=1000, height=500)
    cvar_plot.show()

In [ ]:
r = "PERI"
a = "8572CM"
# m = "CFos"
def check_animal_region(animal_name: str, region_acronym: str, marker=None):
    try:
        sliced_brain = next(animal for group in groups_slices for animal in group if animal_name == animal.name)
    except StopIteration:
        print(f"Can't find region '{region_acronym}' for animal '{animal_name}'")
        return
    sliced_brain = BraiAn.merge_sliced_hemispheres(sliced_brain)
    all_slices_df = sliced_brain.concat_slices()
    slices_per_area = all_slices_df.groupby(all_slices_df.index).count().iloc[:,0]
    markers = sliced_brain.markers if marker is None else [marker]
    brain_avg = BraiAn.AnimalBrain(sliced_brain, mode="avg", hemisphere_distinction=False, use_literature_reuniens=USE_LITERATURE_REUNIENS).data
    brain_std = BraiAn.AnimalBrain(sliced_brain, mode="std", hemisphere_distinction=False, use_literature_reuniens=USE_LITERATURE_REUNIENS).data
    for m in markers:
        marker_avg = brain_avg[m]
        marker_std = brain_std[m]
        print(f"""Summary for brain region '{region_acronym}' of marker '{m}':
            - N slices: {slices_per_area[region_acronym]}
            - Mean: {marker_avg[region_acronym]:.2f} {m}/mm²),
            - S.D.: {marker_std[region_acronym]:.2f} {m}/mm²,
            - Coefficient of Variation: {marker_avg[region_acronym]}
        """)
# for a in groups[-1].dirs:
check_animal_region(a, r) #, m)

In [ ]:
# if you want to see the slices in specific, run this
import pandas as pd
slices = []
try:
    sliced_brain = next(animal for group in groups_slices for animal in group if a == animal.name)
    sliced_brain = BraiAn.merge_sliced_hemispheres(sliced_brain)
    for slice in sliced_brain.slices:
        if r not in slice.markers_density.index:
            continue
        region_densities = slice.markers_density.loc[r].copy()
        region_densities.index += " density"
        region_densities.name = slice.name
        slices.append(region_densities)
except StopIteration:
    print(f"Can't find region '{r}' for animal '{a}'")
pd.concat(slices, axis=1) if len(slices) != 0 else None

In [ ]:
# NOTE: brains are being written WITH Left/Right discrimination
# If you desire to save them without, call AnimalBrain with hemisphere_distinction=False

groups_sum_brains: List[List[BraiAn.AnimalBrain]] = [[BraiAn.AnimalBrain(sliced_brain, mode=BRAINS_AGGREGATION_MODE, use_literature_reuniens=USE_LITERATURE_REUNIENS) for sliced_brain in sliced_brain_list] for sliced_brain_list in groups_slices]
if SAVE_ANIMALS:
    for i in range(len(groups)):
        for animal in groups_sum_brains[i]:
            animal.write_all_brains(data_output_path)

In [ ]:
if REMOVE_HIGH_CV_REGIONS:
    for i, group_slices in enumerate(groups_slices):
        for animal_brain, slices in zip(groups_sum_brains[i], group_slices):
            # TODO: hemisphere_distinction or not?
            cvars = BraiAn.AnimalBrain(slices, mode="cvar", hemisphere_distinction=False, min_slices=0).data
            # TODO: currently there is no differentiation between real markers and overlapping markers.
            # This bad workaround excludes all those markers having a '+' in the name.
            real_markers = [m for m in cvars.markers if "+" not in m]
            disperse_regions = cvars.index[(cvars > CVAR_THRESHOLD)[real_markers].any(axis=1)]
            print(f"removing {len(disperse_regions)}/{len(cvars)} dispersive regions from '{slices.name}'")
            animal_brain.data[~animal_brain.data.index.isin(disperse_regions)]

In [ ]:
animal_groups: List[BraiAn.AnimalGroup] = [BraiAn.AnimalGroup(groups[i].name, groups_sum_brains[i], AllenBrain) for i in range(len(groups))]

In [ ]:
if SAVE_GROUPS:
    for i in range(len(groups)):
        animal_groups[i].to_csv(data_output_path, f"cell_counts_{groups[i].name}.csv", overwrite=True)

In [ ]:
import bgheatmaps as bgh
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import vedo as vd

# vd.embedWindow(None)
vd.embedWindow("k3d") # doesn't spawn a 3rd party window frame for the scene
# vd.embedWindow("itkwidgets")

def make_filename(*ss: str):
    return "_".join((s.replace(' ', '_') for s in ss if s != ""))

def plot_brain(plot_filename: str, plots_output_path: str, brain_data: pd.DataFrame, n: int, title: str, selected_regions: list[str],
                cmin=None, cmax=None, cmap="OrRd", show_text=True):
    heatmaps = []
    hems = ("right", "left")
    # brain_data = brain_data.loc[selected_regions]
    brain_data = brain_data[brain_data.index.isin(selected_regions)]
    # brain_data = brain_data[~brain_data.isna().all(axis=1)]
    if cmin is None:
        cmin = math.floor(brain_data[brain_data != np.inf].min(axis=None))
    if cmax is None:
        cmax = math.ceil(brain_data[brain_data != np.inf].max(axis=None))
    for metric, hem in zip(brain_data.columns, hems):
        metric_data = brain_data[metric]
        dataDict = metric_data.to_dict()

        print(f"Rendering '{metric}' heatmap ({hem} hemisphere)...")
        heatmap = bgh.heatmap(
            dataDict,
            position=5000,
            orientation="frontal",
            thickness=15000,
            title=metric,
            cmap=cmap, # "PuBu",
            vmin=cmin,
            vmax=cmax,
            format="2D",
            hemisphere=hem
        )
        #heatmap.scene.close()
        heatmaps.append(heatmap)
    heatmap1, heatmap2 = heatmaps

    print("depths: ", end="")
    for depth in np.linspace(1500,11000,n):
        # center = heatmap.scene.root.centerOfMass()
        print(f"{depth:.2f}", end="  ")
        s = bgh.slicer.Slicer(depth, "frontal", 100, heatmap1.scene.root) # BraiAn.Plane(center, np.array([0,0,1]), np.array([0,1,0])) # frontal
        heatmap.slicer = s
        projected1, _ = heatmap.slicer.get_structures_slice_coords(heatmap1.regions_meshes, heatmap1.scene.root)
        projected2, _ = heatmap.slicer.get_structures_slice_coords(heatmap2.regions_meshes, heatmap2.scene.root)

        f, ax = plt.subplots(figsize=(9, 9))
        for r, coords in projected1.items():
            name, segment = r.split("_segment_")
            filled_polys = ax.fill(
                coords[:, 0],
                coords[:, 1],
                color=heatmap1.colors[name],
                label=name if segment == "0" and name != "root" else None,
                lw=1,
                ec="k",
                zorder=-1 if name == "root" or heatmap1.colors[name] == [0,0,0] else None,
                alpha=0.3 if name == "root" or heatmap1.colors[name] == [0,0,0] else None,
            )
            if name == "root":
                continue
            (x0, y0), (x1, y1) = filled_polys[0].get_path().get_extents().get_points()
            if show_text:
                ax.text((x0 + x1) / 2, (y0 + y1) / 2, name, ha="center", va="center", fontsize=10, color="black")
        for r, coords in projected2.items():
            name, segment = r.split("_segment_")
            filled_polys = ax.fill(
                coords[:, 0],
                coords[:, 1],
                color=heatmap2.colors[name],
                label=name if segment == "0" and name != "root" else None,
                lw=1,
                ec="k",
                zorder=-1 if name == "root" or heatmap2.colors[name] == [0,0,0] else None,
                alpha=0.3 if name == "root" or heatmap2.colors[name] == [0,0,0] else None,
            )
            if name == "root":
                continue
            (x0, y0), (x1, y1) = filled_polys[0].get_path().get_extents().get_points()
            if show_text:
                ax.text((x0 + x1) / 2, (y0 + y1) / 2, name, ha="center", va="center", fontsize=10, color="white")

        # set title
        ax.set_title(title, fontsize=20, pad=-15)
        for data_column, hem in zip(brain_data.columns, reversed(hems)): # the hemispheres are flipped because the brain is cut front->back, not back->front
            ax.set_title(data_column, loc=hem, y=0, pad=-15)

        # style axes
        ax.invert_yaxis()
        ax.spines["right"].set_visible(False)
        ax.spines["top"].set_visible(False)
        ax.spines["left"].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set(xlabel='', ylabel='')
        ax.set_aspect('equal',adjustable='box')

        # add colorbar
        ax.figure.colorbar(
            mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin=heatmap.vmin, vmax=heatmap.vmax), cmap=heatmap.cmap),
            ax=ax, label=title, fraction=0.046, pad=0.04
        )

        
        plot_filepath = os.path.join(plots_output_path, plot_filename+f"_{depth:05.0f}.svg")
        f.savefig(plot_filepath)
        plt.close(f)
    print("")

In [ ]:
import itertools
import numpy as np

def marker_overlapping_group(group: list[BraiAn.AnimalBrain], marker1: str, marker2: str) -> pd.DataFrame:
    group_overlaps = dict()
    for animal_brain in group:
        animal_brain = BraiAn.AnimalBrain.merge_hemispheres(animal_brain)
        overlaps = animal_brain.overlap_markers(marker1, marker2)
        group_overlaps[overlaps.name] = overlaps.data
    all_animals = pd.concat(group_overlaps, join="outer")
    all_animals = all_animals.reorder_levels([1,0], axis=0)
    ordered_indices = itertools.product(AllenBrain.list_all_subregions("root", mode="depth"), [animal.name for animal in group])
    all_animals = all_animals.reindex(ordered_indices, fill_value=np.nan)
    # commented lines create a DataFrame similar to the one from AnimalGroup
    # all_animals.columns = pd.MultiIndex.from_product((all_animals.columns, (all_animals.columns.name,)))
    # all_animals.columns.names = (None,)*2
    return all_animals

In [ ]:
if "comparison" in config:
    for comp in config["comparison"].values():
        if not isinstance(comp, dict):
            continue
        plots_output_dir = os.path.join(PLOTS_ROOT, comp["dir"])
        os.makedirs(plots_output_dir, exist_ok=True)
        if "groups" in comp and len(comp["groups"]) == 2:
            # group comparison
            right_g, left_g = (animal_groups[i-1] for i in comp["groups"])
            if not right_g.is_comparable(left_g):
                print(f"WARNING: '{right_g}' is not comparable with '{left_g}'!")
                continue
            metric = comp["metric"]
            for marker in right_g.markers:
                if "+" in marker:
                    # TODO: currently there is no differentiation between real markers and overlapping markers.
                    # This bad workaround excludes all those markers having a '+' in the name.
                    continue
                if metric in right_g.get_normalization_methods():
                    groups_diff = pd.concat({
                        right_g.name: right_g.group_by_region(marker=marker, method=metric).mean(), # .apply(lambda x: np.nan if x.isna().all() else np.nanmean(x)), # .data[marker]["Density"].groupby(level=0).mean(),
                        left_g.name:   left_g.group_by_region(marker=marker, method=metric).mean()  # .apply(lambda x: np.nan if x.isna().all() else np.nanmean(x))  # .data[marker]["Density"].groupby(level=0).mean()
                    }, axis=1)
                elif metric.lower() == "overlapping":
                    print(f"WARNING: comparison between groups for metric='Overlapping' not implemented yet!") # needs to specify which markers to overlap
                    continue
                filename = make_filename(metric.lower(), marker, f"{groups_diff.columns[0]}+{groups_diff.columns[1]}")
                plot_brain(filename, plots_output_dir, groups_diff, 12, f"{marker} {metric}", selected_regions, cmap="magma_r", show_text=False)
        elif "markers" in comp and len(comp["markers"]) == 2:
            # marker comparison
            metric = comp["metric"]
            for animal_group, animal_list in zip(animal_groups, groups_sum_brains): 
                right_m, left_m = (animal_group.markers[i-1] for i in comp["markers"])
                if metric in animal_group.get_normalization_methods():
                    markers_diff = pd.concat({
                        right_m: animal_group.group_by_region(marker=right_m, method=metric).mean(),
                        left_m:  animal_group.group_by_region(marker=left_m, method=metric).mean()
                    }, axis=1)
                elif metric.lower() == "overlapping":
                    markers_diff = marker_overlapping_group(animal_list, right_m, left_m) #.groupby(level=0).mean()
                    markers_diff = pd.concat({
                        right_m: markers_diff[right_m].groupby(level=0).mean(),
                        left_m:   markers_diff[left_m].groupby(level=0).mean()
                    }, axis=1)
                filename = make_filename(metric.lower(), animal_group.name, f"{right_m}+{left_m}")
                plot_brain(filename, plots_output_dir, markers_diff, 12, f"{animal_group.name} {metric}", selected_regions, cmap="magma_r", show_text=False)

In [ ]:
import importlib
__imported_modules = sys.modules.copy()
for module_name, module in __imported_modules.items():
    if not module_name.startswith("BraiAn"): # and not module_name.startswith("bgheatmaps"):
        continue
    try:
        # print("reaload:", module_name)
        importlib.reload(module)
    except ModuleNotFoundError:
        continue